In [6]:
import numpy as np
from scipy.optimize import minimize
import random
from stl import mesh
from scipy.spatial import ConvexHull
import trimesh

import os

In [7]:
def esferico_para_cartesiano_elipsoide(r1, r2, r3, pontos):
    xyz = []
    for theta, phi in pontos:
        theta_rad = np.radians(theta)
        phi_rad = np.radians(phi)
        x = r1 * np.sin(phi_rad) * np.cos(theta_rad)
        y = r2 * np.sin(phi_rad) * np.sin(theta_rad)
        z = r3 * np.cos(phi_rad)
        xyz.append([x, y, z])
    return np.array(xyz)

def gerar_pontos_iniciais(n_tipos, pontos_por_tipo):
    pontos = []
    tipos = []
    for tipo, n_pontos in enumerate(pontos_por_tipo):
        for _ in range(n_pontos):
            theta = random.uniform(0, 360)
            phi = random.uniform(0, 180)
            pontos.append([theta, phi])
            tipos.append(tipo)
    return np.array(pontos), tipos


def repulsao_total_elipsoide(variaveis, r1, r2, r3, repulsoes, tipos):
    n_pontos = len(variaveis) // 2
    pontos = np.reshape(variaveis, (n_pontos, 2))  
    cartesianos = esferico_para_cartesiano_elipsoide(r1, r2, r3, pontos)

    energia = 0
    for i in range(n_pontos):
        for j in range(i + 1, n_pontos):
            dist = np.linalg.norm(cartesianos[i] - cartesianos[j])  
            tipo_i = tipos[i]
            tipo_j = tipos[j]
            if (tipo_i, tipo_j) in repulsoes:
                k_ij = repulsoes[(tipo_i, tipo_j)]
            else:
                k_ij = repulsoes[(tipo_j, tipo_i)]  
            energia += k_ij / dist  
    return energia


def otimizar_pontos_elipsoide(r1, r2, r3, n_tipos, pontos_por_tipo, repulsoes):
    pontos_iniciais, tipos = gerar_pontos_iniciais(n_tipos, pontos_por_tipo)
    variaveis_iniciais = pontos_iniciais.flatten()

    resultado = minimize(
        repulsao_total_elipsoide,
        variaveis_iniciais,
        args=(r1, r2, r3, repulsoes, tipos),
        method='L-BFGS-B',
        bounds=[(0, 360) if i % 2 == 0 else (0, 180) for i in range(len(variaveis_iniciais))],
        options={'maxiter': 20000}
    )
    
    pontos_otimizados = np.reshape(resultado.x, (sum(pontos_por_tipo), 2))
    energia_final = repulsao_total_elipsoide(resultado.x, r1, r2, r3, repulsoes, tipos)
    return esferico_para_cartesiano_elipsoide(r1, r2, r3, pontos_otimizados), energia_final



def salvar_em_stl(pontos_3d, nome_arquivo='pontos_otimizados.stl'):
    hull = ConvexHull(pontos_3d)  
    polygon_mesh = mesh.Mesh(np.zeros(hull.simplices.shape[0], dtype=mesh.Mesh.dtype))
    for i, simplex in enumerate(hull.simplices):
        for j in range(3):
            polygon_mesh.vectors[i][j] = pontos_3d[simplex[j], :]
    polygon_mesh.save(nome_arquivo)
    print(f"Arquivo STL salvo como '{nome_arquivo}'")



In [8]:
teste = os.path.isdir("Formas")

if not teste:
   os.makedirs("Formas")

r1, r2, r3 = 15, 15, 15
n_tipos = 5
pontos_por_tipo = [4] * 5 
repulsoes = {  
    (0, 0): 10, 
    (1, 1): 10, 
    (2, 2): 10,  
    (3, 3): 10,
    (4, 4): 10,  
    
    (0, 1): 1,   
    (2, 1): 1,  
    (4, 1): 1,
    (0, 2): 1, 
    (0, 4): 1,
    (3, 2): 1,
    (1, 3): 1,
    (0, 3): 1,
    (4, 2): 1,
    (3, 4): 1,
    (2, 4): 1
}  
n_tentativas = 5

nomes = []
valores = []
for i in range(n_tentativas):
    pontos_otimizados, energia = otimizar_pontos_elipsoide(r1, r2, r3, n_tipos, pontos_por_tipo, repulsoes)
    nome = f"Formas/e{round(energia, 2)}.stl"
    salvar_em_stl(pontos_otimizados, nome)
    nomes.append(nome)
    valores.append(energia)


Arquivo STL salvo como 'Formas/e21.23.stl'
Arquivo STL salvo como 'Formas/e21.16.stl'
Arquivo STL salvo como 'Formas/e21.17.stl'
Arquivo STL salvo como 'Formas/e21.29.stl'
Arquivo STL salvo como 'Formas/e21.16.stl'
